In [16]:
import random
def generate_trials(seed=None):
    """
    Generates a list of trials for a multisensory experiment with non-uniform transitional probabilities.
    
    In the visual modality, there are two EXPictive stimuli (vertical, horizontal) and one neutral stimulus (neutralV).
    The probabilities are as follows (with weights shown in parentheses):
    
      - Under one mapping (Condition A):
          * Horizontal: EXPicted outcome is 45 (weight 3) and UEXicted outcome is 135 (weight 1)
          * Vertical:   EXPicted outcome is 135 (weight 3) and UEXicted outcome is 45 (weight 1)
      - Under the alternate mapping (Condition B):
          * Horizontal: EXPicted outcome is 135 (weight 3) and UEXicted outcome is 45 (weight 1)
          * Vertical:   EXPicted outcome is 45 (weight 3) and UEXicted outcome is 135 (weight 1)
    
    The neutral condition is always balanced:
      - neutralV -> 45 (weight 1) and neutralV -> 135 (weight 1).
    
    The auditory modality is structured analogously:
    
      - Two EXPictive tones (1000Hz, 1600Hz) and one neutral tone (neutralA).
      - Under Condition A (Auditory):
          * 1000Hz: EXPicted outcome is 100 (weight 3) and UEXicted outcome is 160 (weight 1)
          * 1600Hz: EXPicted outcome is 160 (weight 3) and UEXicted outcome is 100 (weight 1)
      - Under Condition B (Auditory):
          * 1000Hz: EXPicted outcome is 160 (weight 3) and UEXicted outcome is 100 (weight 1)
          * 1600Hz: EXPicted outcome is 100 (weight 3) and UEXicted outcome is 160 (weight 1)
      - The neutral condition is balanced:
          * neutralA -> 100 (weight 1) and neutralA -> 160 (weight 1)
    
    Trials are generated by taking the full cross–product of visual and auditory pairs.
    Each unique visual–auditory combination is repeated a number of times equal to the product
    of the visual weight and the auditory weight. Thus, the total number of trials is:
    
         Total visual weight (10) × Total auditory weight (10) = 100 trials.
    
    Parameters:
      seed (int, optional): Seed for the random number generator.
    
    Returns:
      List[dict]: A list of trial dictionaries. Each dictionary contains:
         - 'v_leading', 'v_trailing', 'v_pred'
         - 'a_leading', 'a_trailing', 'a_pred'
    """
    if seed is not None:
        random.seed(seed)
    
    # Randomize visual mapping: choose between two conditions.
    if random.choice([True, False]):
        # Condition A: horizontal->CW high, vertical->CCW high.
        visual_pairs = [
            # Vertical stimulus: high probability for CCW, low for CW.
            (90, 135, 'EXP', 3),
            (90, 45,  'UEX', 1),
            # Horizontal stimulus: high probability for CW, low for CCW.
            (0, 45, 'EXP', 3),
            (0, 135, 'UEX', 1),
        ]
    else:
        # Condition B: horizontal->CCW high, vertical->CW high.
        visual_pairs = [
            # Vertical stimulus: high probability for CW, low for CCW.
            (90, 45, 'EXP', 3),
            (90, 135, 'UEX', 1),
            # Horizontal stimulus: high probability for CCW, low for CW.
            (0, 135, 'EXP', 3),
            (0, 45, 'UEX', 1),
        ]
    # Add balanced neutral visual pairs.
    visual_pairs.append(('neutralV', 45, 'neutral', 2))
    visual_pairs.append(('neutralV', 135, 'neutral', 2))
    
    # Randomize auditory mapping: choose between two conditions.
    if random.choice([True, False]):
        # Condition A: 1000Hz->100Hz high, 1600Hz->160Hz high.
        auditory_pairs = [
            (1000, 100, 'EXP', 3),
            (1000, 160, 'UEX', 1),
            (1600, 160, 'EXP', 3),
            (1600, 100, 'UEX', 1),
        ]
    else:
        # Condition B: 1000Hz->160Hz high, 1600Hz->100Hz high.
        auditory_pairs = [
            (1000, 160, 'EXP', 3),
            (1000, 100, 'UEX', 1),
            (1600, 100, 'EXP', 3),
            (1600, 160, 'UEX', 1),
        ]
    # Add balanced neutral auditory pairs.
    auditory_pairs.append(('neutralA', 100, 'neutral', 0)) # no auditory neutral condition
    auditory_pairs.append(('neutralA', 160, 'neutral', 0))
    
    # Build the cross–product.
    trials = []
    for v_lead, v_target, v_cond, v_weight in visual_pairs:
        for a_lead, a_target, a_cond, a_weight in auditory_pairs:
            count = v_weight * a_weight  # number of repetitions for this combination
            
            if count == 0:
                continue  # skip 

            if count % 2 == 0: # event count of trials for this combination 
                n_zeros = count // 2 # we can have an equal number of target and non-target trials
                n_ones = count // 2
            else: # odd number of trials, we need to randomize the distribution of targets and non-targets
                if random.choice([True, False]): # more non-targets than targets
                    n_zeros = count // 2
                    n_ones = count - n_zeros
                else: # more targets than non-targets
                    n_ones = count // 2
                    n_zeros = count - n_ones

            target_list = [0] * n_zeros + [1] * n_ones # create a list of zeros and ones and shuffle
            random.shuffle(target_list)

            for t in target_list:
                trial = {
                'v_leading': v_lead,
                'v_trailing': v_target,
                'v_pred': v_cond,
                'a_leading': a_lead,
                'a_trailing': a_target,
                'a_pred': a_cond,
                'target': t
             }
                trials.append(trial)
    
    # Shuffle the trial order to randomize sequence.
    random.shuffle(trials)
    return trials

In [36]:
import pandas as pd
trials = generate_trials()
df = pd.DataFrame(trials)
df.groupby(["a_pred", "v_pred"])["target"].sum()

a_pred  v_pred 
EXP     EXP        18
        UEX         7
        neutral    12
UEX     EXP         5
        UEX         3
        neutral     4
Name: target, dtype: int64

In [21]:
df.groupby(["a_pred"])["v_pred"].value_counts()

a_pred  v_pred 
EXP     EXP        36
        neutral    24
        UEX        12
UEX     EXP        12
        neutral     8
        UEX         4
Name: count, dtype: int64

In [15]:
trials

[{'v_leading': 'neutralV',
  'v_trailing': 45,
  'v_pred': 'neutral',
  'a_leading': 1600,
  'a_trailing': 160,
  'a_pred': 'UEX',
  'target': 0},
 {'v_leading': 90,
  'v_trailing': 135,
  'v_pred': 'UEX',
  'a_leading': 1600,
  'a_trailing': 160,
  'a_pred': 'UEX',
  'target': 1},
 {'v_leading': 'neutralV',
  'v_trailing': 135,
  'v_pred': 'neutral',
  'a_leading': 1600,
  'a_trailing': 160,
  'a_pred': 'UEX',
  'target': 0},
 {'v_leading': 0,
  'v_trailing': 45,
  'v_pred': 'UEX',
  'a_leading': 1600,
  'a_trailing': 160,
  'a_pred': 'UEX',
  'target': 0},
 {'v_leading': 'neutralV',
  'v_trailing': 45,
  'v_pred': 'neutral',
  'a_leading': 1000,
  'a_trailing': 160,
  'a_pred': 'EXP',
  'target': 0},
 {'v_leading': 0,
  'v_trailing': 135,
  'v_pred': 'EXP',
  'a_leading': 1600,
  'a_trailing': 100,
  'a_pred': 'EXP',
  'target': 0},
 {'v_leading': 90,
  'v_trailing': 45,
  'v_pred': 'EXP',
  'a_leading': 1000,
  'a_trailing': 100,
  'a_pred': 'UEX',
  'target': 1},
 {'v_leading': 0,
